In [13]:
import MySQLdb
import pandas as pd
from tqdm import trange
import os

In [14]:
df = pd.read_csv("/home/cosbi/financialSite/DB/dynamical_Industry_analysis.csv")
df = df.fillna("NULL")

In [15]:
df

,user,title,date,filename
0,c21221203,IGBT,2018-10-29,IGBT關鍵零件佔電動車成本上看10%.pdf
1,c21221203,手機,2018-11-05,產銷調查 白牌出貨大減壓抑市場旺季表現 4Q18全球平板出貨量將季減2.3%.ppt
2,c21221203,英特爾,2018-11-05,產銷調查 英特爾CPU缺貨問題暫緩 4Q18全球NB出貨將與去年同期持平.ppt
3,c21221203,手機,2018-11-05,產銷調查 3Q18大陸內需市場智慧型手機出貨年減7% 預估4Q18年減逾1成.ppt
4,c21221203,半導體,2018-11-05,產銷調查 4Q18大陸市場智慧型手機AP出貨將季減4.6% 28奈米製程AP比重將續降.ppt
5,c21221203,伺服器,2018-11-05,產銷調查 3Q18全球伺服器出貨低於預期 4Q18將季減9%.ppt
6,c21221203,總經2019,2018-12-03,富邦2019趨勢論壇.pdf
7,c21221203,汽車,2018-12-03,IEK_眺望2019系列_全球電動機車產業現況與商機展望_181116.pdf
8,c21221203,汽車,2018-12-03,IEK_眺望2019系列_汽車零組件產業回顧與展望_181116.pdf
9,c21221203,總經中信,2018-12-03,近期全球股市動盪對台股影響及因應之投資策略(勞退舊)-11152018.ppt


In [16]:
db = MySQLdb.connect(host = "localhost", user = "debian-sys-maint",
                     passwd = "CEMj8ptYHraxNxFt", db = "financial", charset = "utf8")

cursor = db.cursor()

In [17]:
for i in trange(len(df)):
    cursor.execute("INSERT INTO industry_analysis (title, date, filename, username)"
                   " VALUES (%s, %s, %s, %s);", (df["title"][i], df["date"][i], df["filename"][i], df["user"][i]))
    db.commit()

db.close()

100%|███████████████████████████████████████████| 20/20 [00:02<00:00,  9.82it/s]


In [12]:
db.close()

OperationalError: (2006, '')